# Training

Experiments were conducted using:
- `mb_vae_dti/training/run.py`: command line interface for training
- and the scripts in `scripts/training/` for running the experiments

This notebook shows some plots and analysis of the unsupervised pretraining, general DTI training and benchmark fine-tuning results.

In [4]:
from resolve import *

Setting working directory to: /home/robsyc/Desktop/thesis/MB-VAE-DTI


In [ ]:
# Analyzing collected results

from mb_vae_dti.validating.analysis import *

df = load_gridsearch_results("notebooks/results/baseline_kiba_rand/")

df

top_5 = df.sort_values(by="best_val_loss", ascending=True).head(5)
print(round(get_test_averages(top_5), 4))
print(f"Params: {top_5['trainable_params'].mean() / 1000000:.1f}M")

# # # get subsets with desired distinction (encoder_type and aggregator_type)
# with_concat = df[df["config.model.aggregator_type"] == "concat"]
# with_attentive = df[df["config.model.aggregator_type"] == "attentive"]

# # # get 5 best performers
# with_concat_best = with_concat.sort_values(by="best_val_loss", ascending=True).head(5)
# with_attentive_best = with_attentive.sort_values(by="best_val_loss", ascending=True).head(5)

# # # get stats
# print(f"Params: {with_concat_best['trainable_params'].mean() / 1000000:.1f}M")
# print(round(get_test_averages(with_concat_best), 4))
# print("-" * 100)
# print(f"Params: {with_attentive_best['trainable_params'].mean() / 1000000:.1f}M")
# print(round(get_test_averages(with_attentive_best), 4))

# cols_of_interest = ["experiment_name", "trainable_params", "test/real_pKd_mse", "test/real_pKi_mse", "test/real_KIBA_mse", "test/binary_accuracy", "test/binary_f1", "test/binary_auprc"]
# df_test = df[cols_of_interest].copy()
# df_test.columns = [col.replace("test/", "") if col.startswith("test/") else col for col in df_test.columns]
# print(df_test.to_string(index=False, float_format="{:8.4f}".format))

# df = load_gridsearch_results("notebooks/results/train_cold/")
# cols_of_interest = ["experiment_name", "trainable_params", "test/real_pKd_mse", "test/real_pKi_mse", "test/real_KIBA_mse", "test/binary_accuracy", "test/binary_f1", "test/binary_auprc"]
# df_test = df[cols_of_interest].copy()
# df_test.columns = [col.replace("test/", "") if col.startswith("test/") else col for col in df_test.columns]
# print(df_test.to_string(index=False, float_format="{:8.4f}".format))

2025-08-03 16:46:33,824 - mb_vae_dti.validating.analysis - INFO - Loading 64 result files from notebooks/results/baseline_kiba_rand


test/loss              0.1573
test/Y_KIBA_ci         0.8595
test/Y_KIBA_mse        0.1573
test/Y_KIBA_pearson    0.8749
test/Y_KIBA_r2         0.7648
test/Y_KIBA_rmse       0.3966
dtype: float64
Params: 16.9M


In [78]:
top_5

,experiment_name,best_val_loss,best_epoch,trainable_params,val/loss,val/Y_KIBA_ci,val/Y_KIBA_mse,val/Y_KIBA_pearson,val/Y_KIBA_r2,val/Y_KIBA_rmse,...,config.loss.weights,config.data.batch_size,config.data.h5_path,config.data.drug_features,config.data.target_features,config.model.embedding_dim,config.model.hidden_dim,config.model.n_layers,config.model.dropout,config.model.encoder_type
61,baseline_finetune_KIBA_cold_b00c0004,0.333460,50,16841728,0.333460,0.760062,0.333460,0.699008,0.472883,0.688165,...,1_0_0_0,64,data/input/dti.h5torch,FP-Morgan,FP-ESP,1024,512,3,0.1,resnet
28,baseline_finetune_KIBA_cold_b01c0004,0.338392,34,16841728,0.338392,0.753427,0.338392,0.690234,0.464987,0.670500,...,1_0_0_0,32,data/input/dti.h5torch,FP-Morgan,FP-ESP,1024,512,3,0.1,resnet
16,baseline_finetune_KIBA_cold_b00c0001,0.339152,26,12908544,0.339152,0.752964,0.339152,0.690663,0.463886,0.752744,...,1_0_0_0,64,data/input/dti.h5torch,FP-Morgan,FP-ESP,768,512,2,0.1,transformer
29,baseline_finetune_KIBA_cold_b00c0018,0.339814,45,17375488,0.339814,0.753953,0.339814,0.691998,0.462838,0.690825,...,1_0_0_0,64,data/input/dti.h5torch,FP-Morgan,FP-ESP,768,512,3,0.1,transformer
33,baseline_finetune_KIBA_cold_b00c0026,0.340133,26,12908544,0.340133,0.752621,0.340133,0.692200,0.462234,0.697769,...,1_0_0_0,32,data/input/dti.h5torch,FP-Morgan,FP-ESP,768,512,2,0.1,transformer


In [57]:
df.sort_values(by="best_val_loss", ascending=True)[
    ['experiment_name', 'best_val_loss', 'best_epoch', 'test/loss', 'config.data.batch_size',
     'config.model.embedding_dim', 'config.model.hidden_dim', 
     'config.model.n_layers', 'config.model.dropout', 'config.training.learning_rate',
     'config.model.encoder_type']#, 'config.model.aggregator_type']
    ].head(5)
# df.columns

,experiment_name,best_val_loss,best_epoch,test/loss,config.data.batch_size,config.model.embedding_dim,config.model.hidden_dim,config.model.n_layers,config.model.dropout,config.training.learning_rate,config.model.encoder_type
61,baseline_finetune_KIBA_cold_b00c0004,0.333460,50,0.369488,64,1024,512,3,0.1,0.0005,resnet
28,baseline_finetune_KIBA_cold_b01c0004,0.338392,34,0.371440,32,1024,512,3,0.1,0.0005,resnet
16,baseline_finetune_KIBA_cold_b00c0001,0.339152,26,0.366906,64,768,512,2,0.1,0.0005,transformer
29,baseline_finetune_KIBA_cold_b00c0018,0.339814,45,0.371739,64,768,512,3,0.1,0.0005,transformer
33,baseline_finetune_KIBA_cold_b00c0026,0.340133,26,0.371350,32,768,512,2,0.1,0.0005,transformer


In [52]:
df.columns

Index(['experiment_name', 'best_val_loss', 'best_epoch', 'trainable_params',
       'val/loss', 'val/Y_pKd_ci', 'val/Y_pKd_mse', 'val/Y_pKd_pearson',
       'val/Y_pKd_r2', 'val/Y_pKd_rmse', 'test/loss', 'test/Y_pKd_ci',
       'test/Y_pKd_mse', 'test/Y_pKd_pearson', 'test/Y_pKd_r2',
       'test/Y_pKd_rmse', 'total_training_time', 'avg_time_per_epoch',
       'total_epochs', 'config.training.learning_rate',
       'config.training.scheduler', 'config.loss.weights',
       'config.data.batch_size', 'config.data.h5_path',
       'config.data.drug_features', 'config.data.target_features',
       'config.model.embedding_dim', 'config.model.hidden_dim',
       'config.model.n_layers', 'config.model.dropout',
       'config.model.encoder_type'],
      dtype='object')

In [ ]:
# Generating a molecule from the limit distribution

from mb_vae_dti.training.diffusion.utils import *
import json
import torch

json_path = "data/processed/molecular_statistics.json"

stats = json.load(open(json_path))

dataset_name = "drugs_cold"

visualization_tools = MolecularVisualization(stats["general"]["atom_types"])
nodes_dist = DistributionNodes(stats["datasets"][dataset_name]["node_count_distribution"])
limit_dist = PlaceHolder(
    X=torch.tensor(stats["datasets"][dataset_name]["node_marginals"]), 
    E=torch.tensor(stats["datasets"][dataset_name]["edge_marginals"]), 
    y=torch.ones(1) / 1
)

print(f"Limit distribution\nX: {limit_dist.X.shape}\nE: {limit_dist.E.shape}")

n_nodes = nodes_dist.sample_n(1, device="cpu")
print(f"Number of nodes: {n_nodes} (sampled)")

arange = torch.arange(max(n_nodes), device="cpu").unsqueeze(0).expand(1, -1)
node_mask = arange < n_nodes.unsqueeze(1)

print(f"Node mask: {node_mask}")

z_T = sample_discrete_feature_noise(limit_dist=limit_dist, node_mask=node_mask)

print(f"Z_T.X.shape: {z_T.X.shape}")
print(f"Z_T.E.shape: {z_T.E.shape}")

edge_types = torch.argmax(z_T.E, dim=-1)
edge_types = edge_types# - 1
node_types = torch.argmax(z_T.X, dim=-1)


limit_mol = visualization_tools.mol_from_graphs(node_types[0], edge_types[0])

print(f"Successfully created molecule: {limit_mol}")
if limit_mol is not None:
    print(f"Molecule SMILES: {Chem.MolToSmiles(limit_mol)}")
else:
    print("Failed to create valid molecule") 


import rdkit.Chem as Chem
from rdkit.Chem import Draw

Draw.MolToFile(limit_mol, "limit_mol.png")

Limit distribution
X: torch.Size([8])
E: torch.Size([5])
Number of nodes: tensor([27]) (sampled)
Node mask: tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True]])
Z_T.X.shape: torch.Size([1, 27, 8])
Z_T.E.shape: torch.Size([1, 27, 27, 5])
Successfully created molecule: <rdkit.Chem.rdchem.Mol object at 0x757c5cd5a2e0>
Molecule SMILES: C.C.CC.cc1CC(C)(o(c)CC)=n234c(-n(c)n)-c(c-2O3)n-12Oc-2(c)(c)c-4
